## Week 14

## Sets

Collection of elements
* not ordered
* no duplicates

In [2]:
s = { 1,2,3 }
print(s)

{1, 2, 3}


`set()` can be used to convert a list into a set

In [3]:
mylist=list(range(6))
s1=set(mylist)
print(s1)

{0, 1, 2, 3, 4, 5}


In [4]:
str1='mississipi'
list(str1)

['m', 'i', 's', 's', 'i', 's', 's', 'i', 'p', 'i']

In [5]:
set(str1)

{'i', 'm', 'p', 's'}

In [6]:
s2=set(str1)

In [7]:
'p' in s2

True

In [8]:
'q' in s2

False

Looking for an element in a set is (usually) quicker than doing the same to a list

In [9]:
mylist=list(range(10**8))
myset=set(mylist)

**Ex.** write code that checks if  the integers $[10^8,10^8+9]$ are in `mylist` and `myset`, timing the search

In [10]:
from time import time

In [11]:
time()

1588011592.3991663

In [12]:
st=time()

et=time()
print(et-st)

0.0


In [14]:
st=time()
for k in range(10**8,10**8 + 10):
    print(k,k in mylist)
et=time()
print(et-st)

100000000 False
100000001 False
100000002 False
100000003 False
100000004 False
100000005 False
100000006 False
100000007 False
100000008 False
100000009 False
16.617631912231445


In [15]:
st=time()
for k in range(10**8,10**8 + 10):
    print(k,k in myset)
et=time()
print(et-st)

100000000 False
100000001 False
100000002 False
100000003 False
100000004 False
100000005 False
100000006 False
100000007 False
100000008 False
100000009 False
0.00099945068359375


In [16]:
s2

{'i', 'm', 'p', 's'}

In [17]:
for k in s2:
    print(k)

m
s
p
i


### Set operations

In [22]:
set1={'a','b','c','d'}
set2={'a','d','e','f'}

In [23]:
set3=set1 | set2 # union
print(set3)

{'c', 'd', 'e', 'a', 'f', 'b'}


In [24]:
set4 = set1 & set2 # instersection
print(set4)

{'d', 'a'}


In [25]:
set1 - set2 # difference

{'b', 'c'}

In [26]:
set2 - set1

{'e', 'f'}

In [27]:
set2 ^ set1 # symmetric difference

{'b', 'c', 'e', 'f'}

In [29]:
set1.add(1) # adding one element
print(set1)

{'c', 'd', 1, 'a', 'b'}


In [30]:
set1.discard(1) # removing one element
print(set1)

{'c', 'd', 'a', 'b'}


In [31]:
pippo=set1.pop() # remove one element at random, and returns it
print(pippo)
print(set1)

c
{'d', 'a', 'b'}


In [32]:
sa={1,2,3,4,5}
sb={1,4,5}

In [33]:
sb<sa # subset

True

### Note: sets are mutable

In [35]:
sa

{1, 2, 3, 4, 5}

In [36]:
sc=sa
sc.pop()

1

In [37]:
sa

{2, 3, 4, 5}

In [39]:
sa.add([1,2,3]) # only non-mutable object can be in a set

TypeError: unhashable type: 'list'

In [40]:
s={}
type(s)

dict

In [42]:
s=set() # to create an empty set
type(s)

set

### Tombola simulator (very similar to bingo)

* numbers from 1-90
* tables with 15 at random
* every time a number from your table is drawn you remove it from your table.

**Ex.** By Wednesday (use a set to store the numbers of your table)
1. Will create your personal tombola table with 15 numbers at random from $[1,45]$ (`random.sample(set1,n)` will grab 15 numbers at random, w/out repetitions from set1)
2. Write a function `tombola()` that given you table will check if the number drawn is in the table (`input()`) as long as the table is not empty.
3. If the number is in the table it's removed

In [45]:
pippo=input()

Simone


In [46]:
pippo

'Simone'

In [48]:
#tombola()
pippo=input()

5


In [58]:
print('5 was in your table, 12 numbers left')

5 was in your table, 12 numbers left


In [59]:
print('5 was not in your table, 13 numbers left')

5 was not in your table, 13 numbers left


In [51]:
pippo=input()

6


In [52]:
'No more numbers in your table, Tombola!!!'

'No more numbers in your table, Tombola!!!'

In [57]:
import random
random.sample(range(10),2)

[9, 6]

In [60]:
a=set([9,6])

In [62]:
a.discard(9)

In [63]:
a

{6}

In [65]:
type(random.sample([1,2,3],2))

list